In [1]:
import pathlib


def get_data_from_dir_to_dict(data_dir):
  
  data_dir=pathlib.Path(data_dir)

  classes= [str(i).split('/')[-1] for i in list(data_dir.glob('*'))]
  images={}
  for i in classes:
    images[i]=list(test_dir.glob(f'{i}/*'))

  return classes, images



In [2]:
# from pyunpack import Archive
# Archive("vehicles_labeling.rar").extractall(".")
!git clone https://github.com/ahmedramadan96/EALPR.git

Cloning into 'EALPR'...
remote: Enumerating objects: 18740, done.
remote: Counting objects: 100% (12906/12906), done.
remote: Compressing objects: 100% (12903/12903), done.
remote: Total 18740 (delta 12), reused 12892 (delta 3), pack-reused 5834
Receiving objects: 100% (18740/18740), 273.19 MiB | 36.75 MiB/s, done.
Resolving deltas: 100% (32/32), done.
Updating files: 100% (18735/18735), done.


In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet
from glob import glob
from skimage import io
from shutil import copy


In [4]:
path = 'EALPR/EALPR Vechicles dataset/Vehicles Labeling'
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in os.listdir(path):
    filePath=path+"/"+filename
    with open(filePath) as cur:
        info = cur.readline()
        info=info.split()
        if len(info)<5:
          continue
        labels_dict['filepath'].append(filePath)
        labels_dict['xmin'].append(float(info[1]))    #xmin
        labels_dict['xmax'].append(float(info[2]))   #xmax
        labels_dict['ymin'].append(float(info[3]))    #ymin
        labels_dict['ymax'].append(float(info[4]))   #ymax
        cur.close()

In [5]:
df = pd.DataFrame(labels_dict)
df.to_csv('vehicles_labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,EALPR/EALPR Vechicles dataset/Vehicles Labelin...,0.427778,0.580247,0.231481,0.167901
1,EALPR/EALPR Vechicles dataset/Vehicles Labelin...,0.800521,0.616146,0.140625,0.088542
2,EALPR/EALPR Vechicles dataset/Vehicles Labelin...,0.091250,0.570000,0.062500,0.060000
3,EALPR/EALPR Vechicles dataset/Vehicles Labelin...,0.416936,0.762500,0.152104,0.085417
4,EALPR/EALPR Vechicles dataset/Vehicles Labelin...,0.474000,0.510000,0.166667,0.078667


In [6]:
for x in range(len(df['filepath'])):
  if df['filepath'][x]=='EALPR/EALPR Vechicles dataset/Vehicles Labeling/0002.txt':
    print(x)
    break

1622


In [7]:
def find_files(filename, search_path):
   result = []
# Wlaking top-down from the root
   for root, dir, files in os.walk(search_path):
      if filename in files:
         result.append(os.path.join(root, filename))
   return result

In [8]:
df['filepath'][0]
filename = df['filepath'][0]
filename_image =filename.split('/')[3].split('.')[0]+'.txt'
filename_image ,df['filepath'][0]

('1197.txt', 'EALPR/EALPR Vechicles dataset/Vehicles Labeling/1197.txt')

In [9]:
df['filepath'][0]

'EALPR/EALPR Vechicles dataset/Vehicles Labeling/1197.txt'

In [10]:
filename = df['filepath'][0]
def getFilename(filename):
    filename_image =filename.split('/')[3].split('.')[0]+'.jpg'
    if len(find_files(filename_image,'EALPR/EALPR Vechicles dataset/Vehicles'))==0:
      filename_image =filename.split('/')[3].split('.')[0]+'.JPG'
    if len(find_files(filename_image,'EALPR/EALPR Vechicles dataset/Vehicles'))==0:
      filename_image =filename.split('/')[3].split('.')[0]+'.jpeg'
    filepath_image = os.path.join('EALPR/EALPR Vechicles dataset/Vehicles',filename_image)
    return filepath_image
print(getFilename(filename))

EALPR/EALPR Vechicles dataset/Vehicles/1197.jpg


In [11]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10]#random check

['EALPR/EALPR Vechicles dataset/Vehicles/1197.jpg',
 'EALPR/EALPR Vechicles dataset/Vehicles/1885.jpg',
 'EALPR/EALPR Vechicles dataset/Vehicles/0449.jpg',
 'EALPR/EALPR Vechicles dataset/Vehicles/0693.jpg',
 'EALPR/EALPR Vechicles dataset/Vehicles/1483.jpg',
 'EALPR/EALPR Vechicles dataset/Vehicles/1887.jpg',
 'EALPR/EALPR Vechicles dataset/Vehicles/0006.jpg',
 'EALPR/EALPR Vechicles dataset/Vehicles/0079.jpg',
 'EALPR/EALPR Vechicles dataset/Vehicles/1697.jpg',
 'EALPR/EALPR Vechicles dataset/Vehicles/1680.jpg']

In [12]:
file_path = image_path[0] #path of our image N2.jpeg
img = cv2.imread(file_path) #read the image
# xmin-1804/ymin-1734/xmax-2493/ymax-1882 
img = plt.imread(file_path) #Read the image

In [13]:
h=img.shape[0]
w=img.shape[1]

In [14]:
def convert(bx,by,bw,bh):
  xmx=((2*w*bx+w*bw)/2)
  xmn=(2*w*bx-xmx)
  ymx=((2*h*by+h*bh)/2)
  ymn=(2*h*by-ymx)
  return xmn,xmx,ymn,ymx

In [15]:
bx=df['xmin'].iloc[0]
by=df['xmax'].iloc[0]
bw=df['ymin'].iloc[0]
bh=df['ymax'].iloc[0]
xmn,xmx,ymn,ymx=convert(bx,by,bw,bh)

In [16]:
# fig = px.imshow(img)
# fig.update_layout(width=800, height=800,margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure - image with bounding box')
# fig.add_shape(type='rect',x0=xmn, x1=xmx, y0=ymn, y1=ymx, xref='x', yref='y',line_color='cyan')

In [17]:
# n=203
# bx=df['xmin'].iloc[n]
# by=df['xmax'].iloc[n]
# bw=df['ymin'].iloc[n]
# bh=df['ymax'].iloc[n]
# xmn,xmx,ymn,ymx=convert(bx,by,bw,bh)
# file_path = image_path[n] #path of our image N2.jpeg
# img = cv2.imread(file_path)
# fig = px.imshow(img)
# # w,h,l= img.shape
# h,w,l= img.shape
# fig.update_layout(width=w, height=h,margin=dict(l=5, r=5, b=5, t=5),xaxis_title='Figure - image with bounding box')
# fig.add_shape(type='rect',x0=xmn, x1=xmx, y0=ymn, y1=ymx, xref='x', yref='y',line_color='green')


# Read Data

In [18]:
labels = df.iloc[:,1:].values
print(labels)
data = []
output = []


[[0.427778 0.580247 0.231481 0.167901]
 [0.800521 0.616146 0.140625 0.088542]
 [0.09125  0.57     0.0625   0.06    ]
 ...
 [0.374074 0.753695 0.185185 0.17734 ]
 [0.484375 0.682031 0.203125 0.098437]
 [0.518519 0.610185 0.174074 0.087037]]


In [19]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [20]:
image = image_path[2]
img_arr =cv2.imread(image)
# h,w = img_arr.shape
img_arr.shape

(1000, 800, 3)

In [21]:
# from tensorflow.keras.preprocessing.image import load_img, img_to_array
# for ind in range(len(image_path)):
#     image = image_path[ind]
#     img_arr =cv2.imread(image)
#     h,w,l = img_arr.shape
#     # Prepprocesing
#     load_image = load_img(image,target_size=(224,224))
#     load_image_arr = img_to_array(load_image)
#     norm_load_image_arr = load_image_arr/255.0 # Normalization
#     # Normalization to labels
#     bx,by,bw,bh = labels[ind]
#     xmin,xmax,ymin,ymax=convert(bx,by,bw,bh)
#     nxmin,nxmax = xmin/w,xmax/w
#     nymin,nymax = ymin/h,ymax/h
#     label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
#     # Append
#     data.append(norm_load_image_arr)
#     output.append(label_norm)

In [22]:
# mine
# from tensorflow.keras.preprocessing.image import load_img, img_to_array
X=[]
for i in range(len(image_path)):
    image = image_path[i]
    img_arr =cv2.imread(image)
    h,w,l = img_arr.shape
    img_arr= cv2.resize(img_arr,(224,224))
    X.append(img_arr)
    # Prepprocesing
#     load_image_arr = img_to_array(load_image)
#     norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    bx,by,bw,bh = labels[i]
    xmin,xmax,ymin,ymax=convert(bx,by,bw,bh)
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
#     data.append(norm_load_image_arr)
    output.append(label_norm)

Premature end of JPEG file


In [23]:
# Convert data to array
X = np.array(X,dtype=np.float32)/255.0
y = np.array(output,dtype=np.float32)


In [24]:
X.shape ,y.shape

((2087, 224, 224, 3), (2087, 4))

In [ ]:
import ipykernel


: 

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras import layers
import tensorflow as tf

from tensorflow import keras

: 

In [26]:
# Split the data into training and testing set using sklearn.
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.1,random_state=0)
X_train.shape,X_val.shape ,X_test.shape

((1502, 224, 224, 3), (167, 224, 224, 3), (418, 224, 224, 3))

In [27]:
y_train.shape,y_val.shape,y_test.shape

((1502, 4), (167, 4), (418, 4))

#*Model* *Building*

In [29]:
cnn = keras.Sequential([
#     layer input
#     layers.InputLayer(input_shape=(224, 224, 3)),

    # data augmentation
#    tf.keras.layers.RandomContrast(0.2),
#     layers.RandomFlip("horizontal_and_vertical"),
#     layers.RandomRotation(0.2),
#     layers.RandomTranslation(0.2,0.1),
#     layers.RandomZoom(0.3),
#     cnn
  
  layers.Conv2D(512, 3,input_shape=(224, 224, 3),  activation='relu'),
      layers.MaxPooling2D(),

  layers.Conv2D(256, (3,3),  activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, (3,3), activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, (3,3), activation='relu'),
  layers.MaxPooling2D(),
    layers.Dropout(0.2),
    # dense
    layers.Flatten(),

    layers.Dense(1024,activation='relu',name='D0'),
    layers.Dense(512,activation='relu',name='D1'),

    layers.Dense(256, activation='relu',name='D2'),
    layers.Dense(128, activation='relu',name='D3'), 
    layers.Dense(64, activation='relu',name='D4'),
    layers.Dense(4,activation='linear',name='D5'),
    
])
cnn.compile(
    optimizer= 
    
    keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['accuracy']
)
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 512)     14336     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 111, 111, 512)     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 109, 109, 256)     1179904   
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 54, 54, 256)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 52, 52, 128)       295040    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 64)       

In [ ]:
cnn.fit(X_train,y_train,epochs=140,batch_size=32, validation_data=(X_val,y_val))

In [31]:
cnn.evaluate(X_test,y_test)

14/14 [==============================] - 1s 96ms/step - loss: 0.0028 - accuracy: 0.9522


[0.0027899309061467648, 0.9521530866622925]

In [36]:
cnn.save('./CNN_plate_detection.h5')

In [38]:
loaded_model = tf.keras.models.load_model('./CNN_plate_detection.h5')

In [46]:
x= loaded_model.predict(X_test)
x[0] ,y_test[0]

(array([0.34624308, 0.63625723, 0.5643252 , 0.7019336 ], dtype=float32),
 array([0.4000005, 0.6013075, 0.623431 , 0.709205 ], dtype=float32))